<a href="https://colab.research.google.com/github/John-D-Boom/CompVision/blob/main/Copy_of_testing_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
 
# !unzip -qq 'tiny-imagenet-200.zip'
# !pip install torchmetrics
# !pip install timm

In [13]:
!nvidia-smi

Wed Jan 11 16:22:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    47W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

This notebook checks that I can load the dataset, load torch, syncing with GitHub, etc.


In [14]:
import torch
import numpy
import matplotlib.pyplot as plt
from google.colab import drive
import pandas as pd
import os
import shutil
from skimage import io
from skimage.color import gray2rgb

In [15]:
# import zipfile
# with zipfile.ZipFile('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/MLMI/CompVision/')

In [16]:
# df = pd.read_csv('/content/drive/MyDrive/MLMI/CompVision/heatmap.csv')

In [17]:
# # importing the "tarfile" module
# import tarfile

# # open file
# file = tarfile.open('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200-copy.tar.gz')

# # extracting file
# file.extractall('/content/drive/MyDrive/MLMI/CompVision')

# file.close()


In [18]:
def checkFolderSetup(root_dir, expected_num):
    for root, dirs, files in os.walk(root_dir):
      # For each subfolder in the root directory
        for d in dirs:
          # Get the full path to the subfolder
            subfolder_path = os.path.join(root, d)
          # Count the number of entries in the subfolder
            num_entries = len(os.listdir(subfolder_path))
          # Check if the number of entries in the subfolder is equal to n
            try:
                assert num_entries == expected_num 
            except:
                print(f'{root_dir} has error in directory {d}')
    print(f'Checked all folders in {root_dir}')



In [19]:
# test_dir = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/test'
# val_dir =  '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/val'
# train_dir =  '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/train'
# checkFolderSetup(train_dir, 500)
# checkFolderSetup(test_dir, 25)
# checkFolderSetup(val_dir, 25)


In [20]:
# shutil.copytree('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200', '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200-backup')

In [21]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from skimage import io, transform
from sklearn import metrics
import timm
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from torchmetrics.classification import BinaryAUROC

PyTorch Version:  1.13.0+cu116
Torchvision Version:  0.14.0+cu116


In [22]:
class siameseDataset(Dataset):
    
    def __init__(self, csv_file, transform=None, device = "cuda"):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images_frame = pd.read_csv(csv_file, names = ["Image1 Path", "Label1", "Image2 Path", "Label2"])
        train_csv = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/train.csv'
        if(csv_file == train_csv):
            print('is train')
            self.sim_df = self.images_frame[0:80000:2]
            print('sim dim:', len(self.sim_df))
            self.dif_df = self.images_frame[80000:len(self.images_frame):3]
            print('dif dim:', len(self.dif_df))
            self.images_frame = pd.concat([self.sim_df, self.dif_df])
        
        self.transform = transform

    def __len__(self):
        return len(self.images_frame)

    def __getitem__(self, idx):
        
        # print('getting item')
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        train_path = '/content/tiny-imagenet-200/train'
        val_or_test_path = '/content/tiny-imagenet-200/val/images'   
        
        
        img_path1 = self.images_frame.iloc[idx, 0].split('/tiny-imagenet-200/')[1] #Unpacks path and fixes to be compatible with google drive 
        label1 = self.images_frame.iloc[idx, 1]
        
        img_path2 = self.images_frame.iloc[idx, 2].split('/tiny-imagenet-200/')[1] #Unpacks path and fixes to be compatible with google drive
        label2 = self.images_frame.iloc[idx, 3]

        spec_img1 = img_path1.split('/')[-1]
        spec_img2 = img_path2.split('/')[-1]
        
        if 'train' in img_path1:
            class_img1 = spec_img1.split("_")[0]
            class_img2 = spec_img2.split("_")[0]
            img_path1 = os.path.join(train_path, class_img1, "images", spec_img1)
            img_path2 = os.path.join(train_path, class_img2, "images", spec_img2)
        elif 'val' in img_path1 or 'test' in img_path1:
            
            img_path1 = os.path.join(val_or_test_path, spec_img1)
            img_path2 = os.path.join(val_or_test_path, spec_img2)
        else:
            raise ValueError('Bad Image Path. "Test", "Train" "Val" not found.')

        flag = ((label1 == label2) * 2) -1

        # if flag == -1:
        #     print("negative flag loaded")
        # print(img_path1, img_path2)

        image1 = io.imread(img_path1)
        image2 = io.imread(img_path2)

        #Check shapes and enforce RGB

        if (len(image1.shape) != 3):
            # print("bad image1")
            image1 = gray2rgb(image1)
        if (len(image2.shape) != 3):
            # print("bad image2")
            image2 = gray2rgb(image2)
            
        # print('imread successful')
        if self.transform:
            try:
                image1 = self.transform(image1)
                image2 = self.transform(image2)
                # print('transforming')
            except:
                print('ERROR in transforming')
        
        sample = {'image1': image1, 'image2': image2, 'flag': flag}

        return sample

In [23]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet18"

# Number of "classes" in the dataset ***Dimensions of last layer***
num_classes = 128

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs to train for 
num_epochs = 25

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = False

In [24]:
def train_model(model_name, model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    
    loss_history = {"train":[], "val_different":[], "val_similar":[], "epoch_auc":[]}


    best_auc_weights = copy.deepcopy(model.state_dict())
    best_similar_loss = 1e8
    best_different_loss = 1e8

    best_val_auc = 0
    validation_labels = torch.empty(0).to(device)
    validation_pred_distance = torch.empty(0).to(device)


    training_grayscale_errors = 0
    epochs_since_improvement = 0
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        epoch_start = time.time()
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val_different', 'val_similar']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0


            # Iterate over data.
            # print("just before dataloader")
            for batch, obj in enumerate(dataloaders[phase]):
                # print(batch, "/", len(dataloaders[phase]))
                try:
                    # print("trying unpack obj")
                    image1 = obj["image1"]
                    image2 = obj["image2"]
                    flag = obj["flag"]
                except:
                    training_grayscale_errors += 1
                    continue

                image1 = image1.to(device)
                image2 = image2.to(device)
                flag = flag.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.

                    #***ignoring inception model rn
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        # print('running model1')
                        output1 = model(image1)
                        # print('running model2')
                        output2 = model(image2)

                        #Add predictions if in a validation mode
                        if phase == 'val_similar' or phase == 'val_different':
                            distance_func = nn.CosineSimilarity(dim = 1, eps = 1e-6)
                            predicted_similarity = distance_func(output1, output2)
                            # print(predicted_similarity.shape)
                            # print(flag.shape)
                            # assert predicted_similarity.shape == (batch_size,1) #ensure that it is the same size
                            labels = (flag + 1) / 2 #converts to 0 and 1
                            validation_labels = torch.cat((validation_labels,labels))
                            validation_pred_distance = torch.cat((validation_pred_distance, predicted_similarity))

                        #***CRITERION HARDCODED AS COSINEEMBEDDINGLOSS***
                        criterion = nn.CosineEmbeddingLoss(margin=0.0, reduction= 'mean')

                        loss = criterion(output1, output2, flag)


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()



                # statistics
                # print("type of running loss:", type(running_loss))
                running_loss += loss.item() * output1.size(0) #multiplies scalar loss by the number in this batch
                # running_corrects += torch.sum(preds == labels.data)
            
            average_phase_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = float(running_corrects) / len(dataloaders[phase].dataset)
            print('{} Average Loss: {:.4f}'.format(phase, average_phase_loss))
            
            loss_history[phase] += [average_phase_loss]
            
            if phase == 'val_similar':
                #Print time
                epoch_time = time.time() - epoch_start
                print(' Epoch complete in {:.0f}m {:.0f}s'.format(epoch_time // 60, epoch_time % 60))

                #At this point the predictions and ground truth arrays have been fully made
                # print("before metrics")
                # print(validation_labels)
                # print(type(validation_labels))
                # print(validation_labels.shape)

                # print(validation_pred_distance)
                # print(type(validation_pred_distance))
                # print(validation_pred_distance.shape)

                metric = BinaryAUROC(thresholds=None)
                epoch_auc = metric(validation_pred_distance, validation_labels).cpu().numpy()
                # print("type of epoch auc:", type(epoch_auc))
                # print("after metrics")
                print("Epoch AUC: {:.4f}".format(epoch_auc))

                #Append AUC to history
                loss_history["epoch_auc"] += [epoch_auc]
                # print(type(loss_history["epoch_auc"]))

                #Save the logs history
                # print('before declaring dir')
                model_dir = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/{}'.format(model_name)
                if (not os.path.exists(model_dir)):
                    os.mkdir(model_dir)
                # print('before saving to csv')
                # print('loss history type:', type(loss_history))
                # pd.DataFrame(loss_history).to_csv('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/{}/logging_history.txt'.format(model_name), index = False)
                torch.save(loss_history, os.path.join(model_dir, 'logging_history.pkl'))
                print('saved the file')
                #Save model if the AUC is best and reset epoch improvement counter
                print("epoch_auc:", epoch_auc)
                print("best_val_auc:", best_val_auc)
                if epoch_auc > (best_val_auc + 0.01):
                    print("updating epoch accuracy")
                    best_val_auc = epoch_auc
                    epochs_since_improvement = 0
                    best_auc_weights = copy.deepcopy(model.state_dict())

                    #save weights
                    torch.save(best_auc_weights, os.path.join(model_dir, 'best_auc_weights'))
                else:
                    epochs_since_improvement += 1
                    print("This is the {} epoch since improvement".format(epochs_since_improvement))

                    #Early stopping
                    if epochs_since_improvement > 5: 
                        return best_auc_weights, loss_history, model_name #Be Done!

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Auc: {:4f}'.format(best_val_auc))

    # load best model weights
  
    

    return best_auc_weights, loss_history, model_name

In [25]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [26]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0
    if model_name == "resnet18":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet50":
        """ Resnet50_128
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet50_64":
        """ Resnet50_64
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 64)
        input_size = 224

    elif model_name == "resnet50_64":
        """ Resnet50_256
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 256)
        input_size = 224
    
    elif model_name == "resnet101":
        """ Resnet101
        """
        model_ft = models.resnet101(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg16_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299
    
    elif model_name == "vit":
        """Vision Transformer
        """
        model_ft = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
        set_parameter_requires_grad(model_ft, feature_extract)
    elif model_name == "efficientnet":
        """efficientnet_b0
        """
        model_ft = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
        efficient_state_dict_path = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/efficientnet/best_auc_weights'
        efficient_state_dict = torch.load(efficient_state_dict_path)

        model_ft.load_state_dict(copy.deepcopy(efficient_state_dict))
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    else:
        print(model_name)
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
# print(model_ft)

In [27]:
#UPDATED AND WORKING 1/4 5:51
# Data augmentation and normalization for training
# Just normalization for validation
input_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets


train_csv = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/train.csv'
val_different_csv = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/val_different.csv'
val_similar_csv = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/val_similar.csv'

siamese_datasets = {}
siamese_datasets["train"] = siameseDataset(csv_file= train_csv, transform=data_transforms['train'])
siamese_datasets["val_different"] = siameseDataset(csv_file= val_different_csv, transform=data_transforms['val'])
siamese_datasets["val_similar"] = siameseDataset(csv_file= val_similar_csv, transform=data_transforms['val'])

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(siamese_datasets[x], batch_size=batch_size, shuffle=True, num_workers = 12, pin_memory = True) for x in ['train', 'val_different', 'val_similar']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Initializing Datasets and Dataloaders...
is train
sim dim: 40000
dif dim: 42400
cuda:0


In [28]:
# # Send the model to GPU
# model_ft = model_ft.to(device)

# # Gather the parameters to be optimized/updated in this run. If we are
# #  finetuning we will be updating all parameters. However, if we are 
# #  doing feature extract method, we will only update the parameters
# #  that we have just initialized, i.e. the parameters with requires_grad
# #  is True.
# params_to_update = model_ft.parameters()
# print("Params to learn:")
# if feature_extract:
#     params_to_update = []
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             params_to_update.append(param)
#             print("\t",name)
# else:
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             print("\t",name)

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(params_to_update, lr=0.005, momentum=0.9)

In [ ]:
#TRAIN MODEL
# Setup the loss fxn
#***NOT CURRENTLY USING THIS CRITERION. HARD CODED IN TRAIN AS COSINEEMBEDDINGLOSS
import gc
criterion = nn.CrossEntropyLoss()

model_names = ["resnet50_64", "resnet50_256"]
completed = ["efficientnet","resnet18",]
not_doing_anymore = ["resnet101", "vit", "vgg","densenet", ]
for model_name in model_names:
    # try:
    #initialize model
    print(model_name)
    model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
    # Send the model to GPU
    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are 
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    # Observe that all parameters are being optimized

        
    optimizer_ft = optim.SGD(params_to_update, lr=0.005, momentum=0.9)


    # Train and evaluate
    best_auc_weights, loss_history, trained_model_name = train_model(model_name, model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

    model_dir = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/{}'.format(trained_model_name)
    if (not os.path.exists(model_dir)):
        os.mkdir(model_dir)
    #save weights
    torch.save(best_auc_weights, os.path.join(model_dir, 'best_auc_weights'))

    #save training history
    pd.DataFrame(loss_history).to_csv('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/{}/logging_history.txt'.format(trained_model_name), index = False)
    
    model_ft.cpu()
    del model_ft
    gc.collect()
    torch.cuda.empty_cache()
    # del model_ft #ensures that space is made for next model
    
    # except Exception as e:
        # print(model_name, "failed")
        # print(Exception)

resnet50_64


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv3.weight
	 layer1.0.bn3.weight
	 layer1.0.bn3.bias
	 layer1.0.downsample.0.weight
	 layer1.0.downsample.1.weight
	 layer1.0.downsample.1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv3.weight
	 layer1.1.bn3.weight
	 layer1.1.bn3.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer1.2.conv3.weight
	 layer1.2.bn3.weight
	 layer1.2.bn3.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv3.weight
	 layer2.0.bn3.weight
	 layer2.0.bn3.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight

In [ ]:
# Initialize the non-pretrained version of the model used for this run
# scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
# scratch_model = scratch_model.to(device)
# scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
# scratch_criterion = nn.CrossEntropyLoss()
# _,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# Plot the training curves of validation accuracy vs. number 
#  of training epochs for the transfer learning method and
#  the model trained from scratch
ohist = []
# shist = []

ohist = [h.cpu().numpy() for h in hist]
# shist = [h.cpu().numpy() for h in scratch_hist]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

In [ ]:
from skimage import io
path = '/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/train/n03544143/n03544143_435.JPEG'
img = io.imread(path)
plt.imshow(img)
print(img.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
sim_hist = np.loadtxt('/content/drive/MyDrive/MLMI/CompVision/tiny-imagenet-200/models/first_resnet18/sim_hist.txt')
plt.plot(sim_hist)
print(sim_hist)